In [5]:
import numpy as np
from numpy import array
from numpy import hstack
from pickle import dump
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from pyquaternion import Quaternion
from math import floor
import toml

from ipynb.fs.full.V07_Utils import *

In [6]:
######################################################################
# Init Configuration
######################################################################

config_path = 'C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Code/V07_config.toml'
cfg = toml.load(config_path)


######################################################################
# Init Parameters
######################################################################

# define number of input and output sequences for quaternions
n_in_seq_quat = cfg['n_in_seq_quat']
n_out_seq_quat = cfg['n_out_seq_quat']

# define number of input and output sequences for quaternions + gaze 
n_in_seq_quat_gaze = cfg['n_in_seq_quat_gaze']
n_out_seq_quat_gaze = cfg['n_out_seq_quat_gaze']

# define number of input and output sequences for position
n_in_seq_pos = cfg['n_in_seq_pos']
n_out_seq_pos = cfg['n_out_seq_pos']

# define number of input and output sequences for position + gaze 
n_in_seq_pos_gaze = cfg['n_in_seq_pos_gaze']
n_out_seq_pos_gaze = cfg['n_out_seq_pos_gaze']


# define number of time steps to look in the past 
n_steps = cfg['n_steps'] 

# define Look Ahead Time (LAT)
LAT_in_rows = cfg['LAT_in_rows'] #6*16,66ms= 64ms

# Multivariate data preparation

In [7]:
def split_sequences(sequences, n_steps, LAT, n_in_seq, n_out_seq):
    X, y = list(), list()
    
    for i in range(len(sequences)): 
        # find the end of this pattern
        stop_ix = i + n_steps
        shift_ix = i + n_steps + LAT  
        seq_length = len(sequences) - LAT
        
        if stop_ix > seq_length:
            print(stop_ix)
            print(seq_length)
            break
        #seq_x = sequences[i:stop_ix,:n_in_seq]
        #seq_y = sequences[shift_ix-1,-n_out_seq:]
        
        seq_x = sequences[i:stop_ix,n_in_seq:]
        seq_y = sequences[shift_ix-1,:n_out_seq]
        
        
        X.append(seq_x)
        y.append(seq_y)

    return np.array(X), np.array(y)

In [8]:
seq_t = ['1','2','3','4','5','6','7','8','9','10']
seq_e = ['1','2','3','4','5','6','7','8']

seq_x = seq_e[:4]
seq_y = seq_e[-4:]
seq_x , seq_y

(['1', '2', '3', '4'], ['5', '6', '7', '8'])

In [9]:
def batch_splitter(X,y):
    print("hi this is your new splitter")
    len_X = len(X)
    Mod_512 = (len_X)%512
    X = X[:-Mod_512]
    y = y[:-Mod_512]
    
    return X,y

In [10]:
########################## 
# Quaternion 
########################## 

def get_sequence_quat(dataframe, dataset_path_train):

    # define sequences   
    in_seq4 = dataframe[['qw']].to_numpy()
    in_seq5 = dataframe[['qx']].to_numpy()
    in_seq6 = dataframe[['qy']].to_numpy()
    in_seq7 = dataframe[['qz']].to_numpy()
    
    out_seq4 = dataframe[['qw']].to_numpy()
    out_seq5 = dataframe[['qx']].to_numpy()
    out_seq6 = dataframe[['qy']].to_numpy()
    out_seq7 = dataframe[['qz']].to_numpy()
        
    # convert to [rows, columns] structure   
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    in_seq5 = in_seq5.reshape((len(in_seq5), 1))
    in_seq6 = in_seq6.reshape((len(in_seq6), 1))
    in_seq7 = in_seq7.reshape((len(in_seq7), 1))
    
    out_seq4 = out_seq4.reshape((len(out_seq4), 1))
    out_seq5 = out_seq5.reshape((len(out_seq5), 1))
    out_seq6 = out_seq6.reshape((len(out_seq6), 1))
    out_seq7 = out_seq7.reshape((len(out_seq7), 1))

    # horizontally stack columns
    dataset = hstack((in_seq4, in_seq5, in_seq6, in_seq7,out_seq4, out_seq5, out_seq6, out_seq7))
        
    # convert into input/output
    X, y = split_sequences(dataset, n_steps, LAT_in_rows, n_in_seq_quat, n_out_seq_quat)
        
    print("X shape is: " + str(X.shape))
    print("y shape is: " + str(y.shape))
    print("------")
    
    return X,y

In [27]:
########################## 
# Quaternion Delta
########################## 

def get_sequence_quat_delta(dataframe, dataset_path_train):
  
    # define sequences   
    in_seq4 = dataframe[['qw']].to_numpy()
    in_seq5 = dataframe[['qx']].to_numpy()
    in_seq6 = dataframe[['qy']].to_numpy()
    in_seq7 = dataframe[['qz']].to_numpy()
    
    out_seq4 = dataframe[['qw']].to_numpy()
    out_seq5 = dataframe[['qx']].to_numpy()
    out_seq6 = dataframe[['qy']].to_numpy()
    out_seq7 = dataframe[['qz']].to_numpy()
    
    # convert to [rows, columns] structure   
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    in_seq5 = in_seq5.reshape((len(in_seq5), 1))
    in_seq6 = in_seq6.reshape((len(in_seq6), 1))
    in_seq7 = in_seq7.reshape((len(in_seq7), 1))
    
    out_seq4 = out_seq4.reshape((len(out_seq4), 1))
    out_seq5 = out_seq5.reshape((len(out_seq5), 1))
    out_seq6 = out_seq6.reshape((len(out_seq6), 1))
    out_seq7 = out_seq7.reshape((len(out_seq7), 1))

    # horizontally stack columns
    dataset_in = hstack((in_seq4, in_seq5, in_seq6, in_seq7))
    dataset_out = hstack((out_seq4, out_seq5, out_seq6, out_seq7))
    
    # get delta's
    dataset_in_q1 = ([Quaternion(q) for q in dataset_in[:-LAT_in_rows]])
    dataset_in_q2 = ([Quaternion(q) for q in dataset_in[1:]])
    delta_in = np.array([(q1/q2).elements for q1, q2 in zip(dataset_in_q1,dataset_in_q2)])

    dataset_out_q1 = ([Quaternion(q) for q in dataset_out[:]])
    dataset_out_q2 = ([Quaternion(q) for q in dataset_out[LAT_in_rows:]])
    delta_out = np.array([(q1/q2).elements for q1, q2 in zip(dataset_out_q1,dataset_out_q2)])
    
    delta = hstack((delta_in, delta_out))

    # convert into input/output
    X, y = split_sequences(delta, n_steps, LAT_in_rows, n_in_seq_quat, n_out_seq_quat)

    print("X shape is: " + str(X.shape))
    print("y shape is: " + str(y.shape))
    print("------")
    
    return X,y

In [ ]:
########################## 
# Quaternion Delta + Gaze
########################## 

def get_sequence_delta_gaze(dataframe, dataset_path_train):

    # define sequences   
    in_seq2 = dataframe[['Eye X']].to_numpy()
    in_seq3 = dataframe[['Eye Y']].to_numpy()
    
    in_seq4 = dataframe[['qw']].to_numpy()
    in_seq5 = dataframe[['qx']].to_numpy()
    in_seq6 = dataframe[['qy']].to_numpy()
    in_seq7 = dataframe[['qz']].to_numpy()
    
    out_seq4 = dataframe[['qw']].to_numpy()
    out_seq5 = dataframe[['qx']].to_numpy()
    out_seq6 = dataframe[['qy']].to_numpy()
    out_seq7 = dataframe[['qz']].to_numpy()
    
    
    # convert to [rows, columns] structure   
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    in_seq3 = in_seq3.reshape((len(in_seq3), 1))
    
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    in_seq5 = in_seq5.reshape((len(in_seq5), 1))
    in_seq6 = in_seq6.reshape((len(in_seq6), 1))
    in_seq7 = in_seq7.reshape((len(in_seq7), 1))
    
    out_seq4 = out_seq4.reshape((len(out_seq4), 1))
    out_seq5 = out_seq5.reshape((len(out_seq5), 1))
    out_seq6 = out_seq6.reshape((len(out_seq6), 1))
    out_seq7 = out_seq7.reshape((len(out_seq7), 1))

    # horizontally stack columns
    dataset_in_gaze = hstack((in_seq2, in_seq3))
    dataset_in_quat = hstack((in_seq4, in_seq5, in_seq6, in_seq7))
    dataset_out = hstack((out_seq4, out_seq5, out_seq6, out_seq7))
    
    # gaze velocities
    dataset_in_g1 = dataset_in_gaze[:-LAT_in_rows]
    dataset_in_g2 = dataset_in_gaze[1:]
    delta_in_gaze = np.array([g1-g2 for g1, g2 in zip(dataset_in_g1,dataset_in_g2)])
    
    # quaternion velocities
    dataset_in_q1 = ([Quaternion(q) for q in dataset_in_quat[:-LAT_in_rows]])
    dataset_in_q2 = ([Quaternion(q) for q in dataset_in_quat[1:]])
    delta_in_quat = np.array([(q1/q2).elements for q1, q2 in zip(dataset_in_q1,dataset_in_q2)])

    dataset_out_q1 = ([Quaternion(q) for q in dataset_out[:]])
    dataset_out_q2 = ([Quaternion(q) for q in dataset_out[LAT_in_rows:]])
    delta_out = np.array([(q1/q2).elements for q1, q2 in zip(dataset_out_q1,dataset_out_q2)])
    
    delta = hstack((delta_in_quat, delta_in_gaze , delta_out))
    print(delta.shape)

    # convert into input/output
    X, y = split_sequences(delta, n_steps, LAT_in_rows, n_in_seq_quat_gaze, n_out_seq_quat_gaze)
        
    print("X shape is: " + str(X.shape))
    print("y shape is: " + str(y.shape))
    print("------")
    
    return X,y

In [13]:
########################## 
# Position
########################## 

def get_sequence_position(dataframe):

    # define sequences
    in_seq4 = dataframe[['x']].to_numpy()
    in_seq5 = dataframe[['y']].to_numpy()
    in_seq6 = dataframe[['z']].to_numpy()
    
    out_seq4 = dataframe[['x']].to_numpy()
    out_seq5 = dataframe[['y']].to_numpy()
    out_seq6 = dataframe[['z']].to_numpy()

    # convert to [rows, columns] structure   
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    in_seq5 = in_seq5.reshape((len(in_seq5), 1))
    in_seq6 = in_seq6.reshape((len(in_seq6), 1))
    
    out_seq4 = out_seq4.reshape((len(out_seq4), 1))
    out_seq5 = out_seq5.reshape((len(out_seq5), 1))
    out_seq6 = out_seq6.reshape((len(out_seq6), 1))

    # horizontally stack columns
    dataset = hstack((in_seq4, in_seq5, in_seq6, out_seq4, out_seq5, out_seq6))
    
    # convert into input/output
    X, y = split_sequences(dataset, n_steps, LAT_in_rows, n_in_seq_pos, n_out_seq_pos)

    print("X shape is: " + str(X.shape))
    print("y shape is: " + str(y.shape))
    print("------")
    
    return X,y

In [14]:
########################## 
# Position Delta
########################## 

def get_sequence_position_delta(dataframe, dataset_path_train):
  
    # define sequences
    in_seq4 = dataframe[['x']].to_numpy()
    in_seq5 = dataframe[['y']].to_numpy()
    in_seq6 = dataframe[['z']].to_numpy()
    
    out_seq4 = dataframe[['x']].to_numpy()
    out_seq5 = dataframe[['y']].to_numpy()
    out_seq6 = dataframe[['z']].to_numpy()
    
    # convert to [rows, columns] structure   
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    in_seq5 = in_seq5.reshape((len(in_seq5), 1))
    in_seq6 = in_seq6.reshape((len(in_seq6), 1))
    
    out_seq4 = out_seq4.reshape((len(out_seq4), 1))
    out_seq5 = out_seq5.reshape((len(out_seq5), 1))
    out_seq6 = out_seq6.reshape((len(out_seq6), 1))

    # horizontally stack columns
    dataset_in = hstack((in_seq4, in_seq5, in_seq6))
    dataset_out = hstack((out_seq4, out_seq5, out_seq6))
    
    dataset_in_q1 = dataset_in[:-LAT_in_rows]
    dataset_in_q2 = dataset_in[1:]
    delta_in = np.array([pos1 - pos2 for pos1, pos2 in zip(dataset_in_q1,dataset_in_q2)])

    dataset_out_q1 = dataset_out[:]
    dataset_out_q2 = dataset_out[LAT_in_rows:]
    delta_out = np.array([pos1 - pos2 for pos1, pos2 in zip(dataset_out_q1,dataset_out_q2)])
    
  
    
    delta = hstack((delta_in, delta_out))

    # convert into input/output
    X, y = split_sequences(delta, n_steps, LAT_in_rows, n_in_seq_pos, n_out_seq_pos)

    print("X shape is: " + str(X.shape))
    print("y shape is: " + str(y.shape))
    print("------")
    
    return X,y

# Training 

In [15]:
########################## 
# Quaternion Delta
########################## 

def prepare_training_data_quat_delta(dataset_path_train):
 
    X_train_result = [] 
    y_train_result = [] 
    
    X_val_result = [] 
    y_val_result = [] 

    for csv in get_csv_files(dataset_path_train, "1_special_train_contin"):
        csv_df = load_df(csv)
        X_train, y_train = get_sequence_quat_delta(csv_df, dataset_path_train)
        
    for csv in get_csv_files(dataset_path_train, "1_special_test_contin"):
        csv_df = load_df(csv)
        X_val, y_val = get_sequence_quat_delta(csv_df, dataset_path_train)
        
    return X_train, X_val, y_train, y_val

In [16]:
########################## 
# Quaternion
########################## 

def prepare_training_data_quat(dataset_path_train):
 
    X_train_result = [] 
    y_train_result = [] 
    
    X_val_result = [] 
    y_val_result = [] 

    for csv in get_csv_files(dataset_path_train, "1_special_train_contin"):
        print(csv)
        csv_df = load_df(csv)
        X_train, y_train = get_sequence_quat_delta(csv_df, dataset_path_train)
        
    for csv in get_csv_files(dataset_path_train, "1_special_test_contin"):
        print(csv)
        csv_df = load_df(csv)
        X_val, y_val = get_sequence_quat_delta(csv_df, dataset_path_train)
        
    return X_train, X_val, y_train, y_val

In [17]:
########################## 
# Position Delta
########################## 

def prepare_training_data_pos_delta(dataset_path_train):
 
    X_train_result = [] 
    y_train_result = [] 
    
    X_val_result = [] 
    y_val_result = [] 

    for csv in get_csv_files(dataset_path_train, "1_special_train_original"):
        csv_df = load_df(csv)
        #csv_df = pd.read_csv(csv, delimiter =";")
        X_train, y_train = get_sequence_position_delta(csv_df, dataset_path_train)
                
    for csv in get_csv_files(dataset_path_train, "1_special_test_original"):
        csv_df = load_df(csv)
        X_val, y_val = get_sequence_position_delta(csv_df, dataset_path_train)
        
    return X_train, X_val, y_train, y_val

In [18]:
########################## 
# Position
########################## 

def prepare_training_data_pos(dataset_path_train):
 
    X_train_result = [] 
    y_train_result = [] 
    
    X_val_result = [] 
    y_val_result = [] 
    
    for csv in get_csv_files(dataset_path_train, "1_special_train_original"):
        csv_df = load_df(csv)
        #csv_df = pd.read_csv(csv, delimiter =";")
        X_train, y_train = get_sequence_position(csv_df)
                
    for csv in get_csv_files(dataset_path_train, "1_special_test_original"):
        csv_df = load_df(csv)
        X_val, y_val = get_sequence_position(csv_df)
        
    return X_train, X_val, y_train, y_val

In [19]:
########################## 
# Quaternion Delta + Gaze Delta
########################## 

def prepare_training_data_quat_gaze_delta(dataset_path_train):
 
    X_train_result = [] 
    y_train_result = [] 
    
    X_val_result = [] 
    y_val_result = [] 

    for csv in get_csv_files(dataset_path_train, "1_gaze_train_contin"):
        csv_df = load_df(csv)
        X_train, y_train = get_sequence_delta_gaze(csv_df, dataset_path_train)
        
    for csv in get_csv_files(dataset_path_train, "1_gaze_test_contin"):
        csv_df = load_df(csv)
        X_val, y_val = get_sequence_delta_gaze(csv_df, dataset_path_train)
        
    return X_train, X_val, y_train, y_val

# Testing

In [20]:
def prepare_testing_data_quat_special_delta(dataset_path_train):
 
    X_train_result = [] 
    y_train_result = [] 
    
    csv_df = load_df(dataset_path_train)
    X, y = get_sequence_quat_delta(csv_df, dataset_path_train)        

    return X, y

In [21]:
def prepare_testing_data_quat_special(dataset_path_train):
 
    X_train_result = [] 
    y_train_result = [] 
    
    csv_df = load_df(dataset_path_train)
    X, y = get_sequence_quat_delta(csv_df, dataset_path_train)        
        
    return X, y

In [22]:
def prepare_testing_data_pos_special_delta(dataset_path_train):
 
    X_train_result = [] 
    y_train_result = [] 
    
    csv_df = load_df(dataset_path_train)
    X, y = get_sequence_position_delta(csv_df, dataset_path_train)        
        
    return X, y

In [23]:
def prepare_testing_data_pos_special(dataset_path_train):
 
    X_train_result = [] 
    y_train_result = [] 
    
    csv_df = load_df(dataset_path_train)
    X, y = get_sequence_position(csv_df)        
        
    return X, y